In [8]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Model,Sequential
from tensorflow.keras.layers import Concatenate,Dense,Input, Conv2D, MaxPooling2D, Flatten, Dense,concatenate,GlobalAveragePooling2D

# ANN

### Type 1

In [2]:
# Define first ANN model
input_layer = keras.Input(shape=(10,))

x1 = Dense(64, activation='relu')(input_layer)
x1 = Dense(32, activation='relu')(x1)
output_1 = Dense(1, activation='sigmoid')(x1)
model_1 = Model(inputs=input_layer, outputs=output_1)

# Define second ANN model
x2 = Dense(64, activation='relu')(input_layer)
x2 = Dense(32, activation='relu')(x2)
output_2 = Dense(1, activation='sigmoid')(x2)
model_2 = Model(inputs=input_layer, outputs=output_2)

# Merge the outputs of the two models
merged_outputs = concatenate([output_1, output_2])

# Define a new model with the merged outputs
x = Dense(64, activation='relu')(merged_outputs)
x = Dense(32, activation='relu')(x)
output = Dense(1, activation='sigmoid')(x)
combined_model = Model(inputs=input_layer, outputs=output)

combined_model.summary()

Model: "model_2"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 10)]         0           []                               
                                                                                                  
 dense (Dense)                  (None, 64)           704         ['input_1[0][0]']                
                                                                                                  
 dense_3 (Dense)                (None, 64)           704         ['input_1[0][0]']                
                                                                                                  
 dense_1 (Dense)                (None, 32)           2080        ['dense[0][0]']                  
                                                                                            

### Type 2

In [3]:
# Define first ANN model
input_1 =Input(shape=(10,))

x1 = Dense(64, activation='relu')(input_1)
x1 = Dense(32, activation='relu')(x1)
output_1 = Dense(1, activation='sigmoid')(x1)
model_1 = Model(inputs=input_1, outputs=output_1)

# Define second ANN model
input_2 = Input(shape=(20,))

x2 = Dense(64, activation='relu')(input_2)
x2 = Dense(32, activation='relu')(x2)
output_2 = Dense(1, activation='sigmoid')(x2)
model_2 = Model(inputs=input_2, outputs=output_2)

# Combine the two models
combined_input = concatenate([output_1, output_2])
x = Dense(64, activation='relu')(combined_input)
x = Dense(32, activation='relu')(x)
output = Dense(1, activation='sigmoid')(x)
combined_model = Model(inputs=[input_1, input_2], outputs=output)

combined_model.summary()

Model: "model_5"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_2 (InputLayer)           [(None, 10)]         0           []                               
                                                                                                  
 input_3 (InputLayer)           [(None, 20)]         0           []                               
                                                                                                  
 dense_9 (Dense)                (None, 64)           704         ['input_2[0][0]']                
                                                                                                  
 dense_12 (Dense)               (None, 64)           1344        ['input_3[0][0]']                
                                                                                            

# CNN

#### Type1

In [4]:
inp = Input(shape=(224,224,3))

#1st conv model
model1 = Sequential()
#conv1
model1.add(Conv2D(6,(5,5),activation='relu'))
model1.add(MaxPooling2D(strides= (2,2)))

#2nd conv model
model2 = Sequential()
#conv1
model2.add(Conv2D(80,(5,5),activation='relu'))
model2.add(MaxPooling2D(strides= (2,2)))

#combaine two model
combined = Concatenate()([model1(inp), model2(inp)])
combined = Flatten()(combined)

#output
outputs = Dense(3,activation="softmax")(combined)

my_model  = Model(inp, outputs)

my_model.summary()

Model: "model_6"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_4 (InputLayer)           [(None, 224, 224, 3  0           []                               
                                )]                                                                
                                                                                                  
 sequential (Sequential)        (None, 110, 110, 6)  456         ['input_4[0][0]']                
                                                                                                  
 sequential_1 (Sequential)      (None, 110, 110, 80  6080        ['input_4[0][0]']                
                                )                                                                 
                                                                                            

#### Type 2

In [5]:
# Define first CNN model
model_1 = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(32, 32, 3)),
    MaxPooling2D((2, 2)),
    Flatten(),
    Dense(64, activation='relu'),
    Dense(10)
])

# Define second CNN model
model_2 = Sequential([
    Conv2D(64, (3, 3), activation='relu', input_shape=(32, 32, 3)),
    MaxPooling2D((2, 2)),
    Flatten(),
    Dense(64, activation='relu'),
    Dense(10)
])

# Combine the two models using averaging
input_layer = Input(shape=(32, 32, 3))
output_1 = model_1(input_layer)
output_2 = model_2(input_layer)
merged_outputs = keras.layers.average([output_1, output_2])

# Define the ensembled model
ensembled_model = Model(inputs=input_layer, outputs=merged_outputs)

ensembled_model.summary()


Model: "model_7"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_5 (InputLayer)           [(None, 32, 32, 3)]  0           []                               
                                                                                                  
 sequential_2 (Sequential)      (None, 10)           462410      ['input_5[0][0]']                
                                                                                                  
 sequential_3 (Sequential)      (None, 10)           924106      ['input_5[0][0]']                
                                                                                                  
 average (Average)              (None, 10)           0           ['sequential_2[0][0]',           
                                                                  'sequential_3[0][0]']     

# Transferlearning with CNN Model

In [6]:
from tensorflow.keras.applications import ResNet50, MobileNet

### Type1

In [14]:
## input shape must be same
inp = Input(shape=(224,224,3))

#1st transfer learning model
model1 = ResNet50(include_top=False, weights='imagenet')

for layer in model1.layers:
    layer.trainable = False 

#2nd transfer learning model
model2 = MobileNet(include_top=False, weights='imagenet')

for layer in model2.layers:
    layer.trainable = False 

#combine two model
combined = Concatenate()([model1(inp), model2(inp)])
combined = Flatten()(combined)

#outputlayer
outputs = Dense(3,activation="softmax")(combined)

#finaly combaine inp with output
my_model  = Model(inp, outputs)

my_model.summary()
#plot_model(my_model, show_shapes=True)

Model: "model_10"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_12 (InputLayer)          [(None, 224, 224, 3  0           []                               
                                )]                                                                
                                                                                                  
 resnet50 (Functional)          (None, None, None,   23587712    ['input_12[0][0]']               
                                2048)                                                             
                                                                                                  
 mobilenet_1.00_224 (Functional  (None, None, None,   3228864    ['input_12[0][0]']               
 )                              1024)                                                      

### Type 2

In [17]:
# Define the simple CNN model
cnn_model = Sequential()
cnn_model.add(Conv2D(filters=32, kernel_size=(3, 3), activation='relu', input_shape=(224, 224, 3)))
cnn_model.add(MaxPooling2D(pool_size=(2, 2)))
cnn_model.add(Conv2D(filters=64, kernel_size=(3, 3), activation='relu'))
cnn_model.add(MaxPooling2D(pool_size=(2, 2)))
cnn_model.add(Conv2D(filters=64, kernel_size=(3, 3), activation='relu'))

cnn_model.add(Flatten())
cnn_model.add(Dense(units=64, activation='relu'))


# Define the ResNet50 model with its pre-trained weights
resnet_model = ResNet50(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
resnet_output = resnet_model.output
resnet_output = Flatten()(resnet_output)

# Concatenate the outputs of both models
merged_output = concatenate([cnn_model.output, resnet_output])

# Add some dense layers with a sigmoid output
merged_output = Dense(256, activation='relu')(merged_output)
merged_output = Dense(128, activation='relu')(merged_output)
merged_output = Dense(1, activation='sigmoid')(merged_output)

# Define the final model
final_model = Model(inputs=[cnn_model.input, resnet_model.input], outputs=merged_output)

final_model.summary()

Model: "model_12"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_16 (InputLayer)          [(None, 224, 224, 3  0           []                               
                                )]                                                                
                                                                                                  
 conv1_pad (ZeroPadding2D)      (None, 230, 230, 3)  0           ['input_16[0][0]']               
                                                                                                  
 conv1_conv (Conv2D)            (None, 112, 112, 64  9472        ['conv1_pad[0][0]']              
                                )                                                                 
                                                                                           

### Type 3

In [20]:
def create_model(input_shape):
    inputs = Input(shape=input_shape)

    x = Conv2D(filters=32, kernel_size=(3, 3), padding='same', activation='relu')(inputs)
    x = MaxPooling2D(pool_size=(2, 2))(x)

    x = Conv2D(filters=64, kernel_size=(3, 3), padding='same', activation='relu')(x)
    x = MaxPooling2D(pool_size=(2, 2))(x)

    x = Conv2D(filters=64, kernel_size=(3, 3), padding='same', activation='relu')(x)

    x = Flatten()(x)
    x = Dense(units=64, activation='relu')(x)
    outputs = Dense(units=10, activation='softmax')(x)

    model = Model(inputs=inputs, outputs=outputs)

    return model

# Define the simple CNN model
input_shape = (28, 28, 1)
simple_cnn_model = create_model(input_shape)

# Define the ResNet50 model with its pre-trained weights
resnet_model = ResNet50(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
resnet_output = resnet_model.output
resnet_output = Flatten()(resnet_output)

# Concatenate the outputs of both models
merged_output = concatenate([simple_cnn_model.output, resnet_output])

# Add some dense layers with a sigmoid output
merged_output = Dense(256, activation='relu')(merged_output)
merged_output = Dense(128, activation='relu')(merged_output)
merged_output = Dense(1, activation='sigmoid')(merged_output)

# Define the final model
final_model = Model(inputs=[simple_cnn_model.input, resnet_model.input], outputs=merged_output)

final_model.summary()

Model: "model_16"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_20 (InputLayer)          [(None, 224, 224, 3  0           []                               
                                )]                                                                
                                                                                                  
 conv1_pad (ZeroPadding2D)      (None, 230, 230, 3)  0           ['input_20[0][0]']               
                                                                                                  
 conv1_conv (Conv2D)            (None, 112, 112, 64  9472        ['conv1_pad[0][0]']              
                                )                                                                 
                                                                                           